In [ ]:
import pandas as pd
import numpy as np
import fiona
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point

# Load data and only select relevant cases
data = pd.read_csv('..//data//created//311_cases_clean.csv')
data['Opened'] = pd.to_datetime(data['Opened'])
data = data[data['Opened']>'2016-05-01']

# Create list and path to shapefile
block_list = []
shape_loc = 'Census 2000_ Blocks for San Francisco (no water)// \
             geo_export_10e804e3-5dd0-4986-9466-956443c0fd64.shp'

for index, row in data.iterrows():
    
    # Open shapefile and get first record
    c_file = fiona.open("..//data//sf_open//{}".format(shape_loc))
    pol = c_file.next()
    geom = shape(pol['geometry'])

    # Establish first test point
    point_test = Point((row['Longitude'],row['Latitude']))
    
    try:
        
        # Loop through each case and grab the block dictionary associated with it
        while geom.contains(point_test) == False:
            pol = c_file.next()
            geom = shape(pol['geometry'])
            
        # Append to list when match is found
        block_list.append(dict(pol['properties']))
        
    except:
        
        # Append a dict with null values if none is found
        block_list.append({k : np.nan for k in dict(pol['properties']).keys()})
    
    # Close the shapefile
    c_file.close()
    
# Create dataframe with all matches
block_df = pd.DataFrame(block_list)
block_df['CaseID'] = list(data['CaseID'])

block_df.to_csv('..//data//created//caseid_block.csv', index = False)

# create list and path to shapefile
block_list = []
c_file = fiona.open("..//data//sf_open//{}".format(shape_loc))
pol = c_file.next()

for i in range(0,5791):
    
    geom = shape(pol['geometry'])
    block_list.append(dict(pol['properties']))
    pol = c_file.next()
    
block_df = pd.DataFrame(block_list)
block_df.to_csv('..//data//created//all_blocks.csv', index=False)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
